In [1]:
import openai
from IPython.display import Image
from typing import Literal, Union
import re
from src.text.open_ai import OpenAITextGeneration, OpenAITextModelConfig

In [2]:
text_model = OpenAITextGeneration()

In [3]:
GET_COMICS = """This is a bot to write long stories for children.
    
Context: The bot's stories usually have a very important lesson for children. Today the bot will talk them about {prompt}

Story:
Once upon a time"""

prompt = 'A child in the forest'
comics = text_model.text2text(GET_COMICS.format(prompt=prompt))
comics = "Once upon a time " + comics
print(comics)


Once upon a time , there lived a young child in a small village. This child had a passion for exploring the forest and often ventured deep into the woods, far away from their village. One day, while exploring, the child stumbled upon a mysterious cave that seemed to have been forgotten by time. 

The child ventured inside and discovered that the cave was filled with beautiful crystals and gems of all colors. The child was mesmerized by the sparkling jewels and continued to explore. Eventually, the child found a secret door that led to an even bigger room filled with even more gems and jewels. 

The child was excited and filled their pockets with the precious stones, not knowing how valuable they were. As the child left the cave, they heard a loud voice calling out to them. The voice belonged to an old wizard who had been watching the child from the shadows. The wizard informed the child that the gems and jewels were sacred and belonged to the forest. 

The wizard explained that the gem

In [4]:
GET_SPLIT = """This is a bot to split stories in principal frames.
    
Story:
Manuel was a very curious little boy. One day, he saw a sign that said "Beware of the forest" and he wanted to see what was inside. So he went in.

Answer:
1-Manuel was a very curious little boy. One day, he saw a sign that said "Beware of the forest"
2-He wanted to see what was inside. So he went in.

Story:
Manuel saw all kinds of animals in the forest. He saw a deer, a rabbit, and even a bear! He was so excited to see all of these animals.
But then, Manuel saw a big, scary snake. The snake was coiled up and ready to strike. Manuel was so scared that he ran out of the forest as fast as he could.

Answer:
1-Manuel saw all kinds of animals in the forest.
2-He saw a deer.
3-He saw a rabbit.
4-He even saw a bear!
5-He was so excited to see all of these animals.
6-But then, ....
7-Manuel saw a big, scary snake. The snake was coiled up and ready to strike.
8-Manuel was so scared that he ran out of the forest as fast as he could.

Story:
When he got home, he was so happy to be safe. He learned that it is always important to listen to warnings and to be careful when exploring new places.

Answer:
1-When he got home, he was so happy to be safe.
2-He learned that it is always important to listen to warnings and to be careful when exploring new places.

Story:
Manuel was a very curious little boy. One day, he saw a sign that said "Beware of the forest" and he wanted to see what was inside. So he went in. 
Manuel saw all kinds of animals in the forest. He saw a deer, a rabbit, and even a bear! He was so excited to see all of these animals. 
But then, Manuel saw a big, scary snake. The snake was coiled up and ready to strike. Manuel was so scared that he ran out of the forest as fast as he could.
When he got home, he was so happy to be safe. He learned that it is always important to listen to warnings and to be careful when exploring new places.
 
Answer:
1-Manuel was a very curious little boy. One day, he saw a sign that said "Beware of the forest"
2-He wanted to see what was inside. So he went in.
3-Manuel saw all kinds of animals in the forest.
4-He saw a deer.
5-He saw a rabbit.
6-He even saw a bear!
7-He was so excited to see all of these animals.
8-But then, ....
9-Manuel saw a big, scary snake. The snake was coiled up and ready to strike.
10-Manuel was so scared that he ran out of the forest as fast as he could.
11-When he got home, he was so happy to be safe.
12-He learned that it is always important to listen to warnings and to be careful when exploring new places.

Story:
{story}

Answer:
1-"""

splits = '1-' + text_model.text2text(GET_SPLIT.format(story = comics))

print(splits)
frames = re.split(r'\d+-\s*', splits.strip())

1-Once upon a time , there lived a young child in a small village. This child had a passion for exploring the forest and often ventured deep into the woods, far away from their village.
2-One day, while exploring, the child stumbled upon a mysterious cave that seemed to have been forgotten by time.
3-The child ventured inside and discovered that the cave was filled with beautiful crystals and gems of all colors. 
4-The child was mesmerized by the sparkling jewels and continued to explore.
5-Eventually, the child found a secret door that led to an even bigger room filled with even more gems and jewels. 
6-The child was excited and filled their pockets with the precious stones, not knowing how valuable they were. 
7-As the child left the cave, they heard a loud voice calling out to them. The voice belonged to an old wizard who had been watching the child from the shadows. 
8-The wizard informed the child that the gems and jewels were sacred and belonged to the forest. 
9-The wizard expla

In [5]:
GET_CHARACTERS = """Detect each characters in a story. If the characters has a name I want to know its name. 
But if the characters doesn't have a name use its sex or its species to make a random name, as example Man 1 or Bear 1

Story: {story}

Character and Randomly generated's name list divided by commas:["""

characters = text_model.text2text(GET_CHARACTERS.format(story=comics))
print('[', characters)

chars = [line.strip() for line in characters[:-1].split(',')]

[ Child, Old Wizard]


In [6]:
GET_DESCRIPTIONS = """I gave you s story and a list of characters in it. Give me a very detail physical description for each characters 

Story: {story}

Character's name list divided by commas: [{characters}

Descriptions:
{first_name}:"""

descriptions = text_model.text2text(GET_DESCRIPTIONS.format(story=comics, characters=characters, first_name = chars[0]))
descriptions = chars[0] + ':' + descriptions 

print(descriptions) 

dd = dict(line.strip().split(':') for line in descriptions.split('\n') if ':' in line)
assert len(dd) == len(chars)


Child:The child was a young person, most likely a preteen or early teenager. They had dark brown hair and kind brown eyes. They were wearing a simple outfit with a white t-shirt, jeans and sneakers. 

Old Wizard: The old wizard was a mysterious figure with a long white beard and wearing a long robe. He had a kind face and piercing blue eyes that seemed to look right into the child's soul. He had a staff in his hand and was wearing a pointed hat.


In [7]:
GET_IMAGE_PROMPT = """I will give you a story and the list of story's characters. I will also give you a frames of the story.
Give a prompt for a text to image generative model with an image description. 
In this prompt when you will reference to some characters change this reference for its name in the list between parenthesis.
Don't use pronouns nor literal references. 

Story:
A young was a very curious little boy. One day, he saw a sign that said "Beware of the forest" and he wanted to see what was inside. So he went in.

Characters: [Child 1, Rabbit, Bear]

Frames:A young was a very curious little boy. One day, he saw a sign that said "Beware of the forest"

Image Prompt:(Child) is watching a sign front of a forest.

Story:
Manuel saw all kinds of animals in the forest. He saw a rabbit.

Characters: [Child 1, Rabbit, Bear]

Frames:He saw a rabbit

Image Prompt:(Child) is watching the (Rabbit) in the forest.

Story: 
{story}

Characters:[{characters}]

FRAME:{frame}

Image Prompts:"""

def auto_prompting(frame):
    return text_model.text2text(GET_IMAGE_PROMPT.format(story=comics, characters=','.join(dd.keys()), frame = frame))

prompts = [auto_prompting(f) for f in frames]

for i, p in enumerate(prompts):
    print(i, p)

0 (Child) is returning a precious stone to the (Old Wizard) in the forest.
1 (Child) is standing outside the entrance of a mysterious cave, looking up at an (Old Wizard) standing in the shadows.
2 (Child) is standing in front of a mysterious cave, filled with colorful gems and jewels, while (Old Wizard) is watching in the shadows.
3 (Child) is exploring a mysterious cave filled with beautiful crystals and gems of all colors.
4 (Child) is exploring a mysterious cave filled with beautiful crystals and gems.
5 (Child) is standing in front of a secret door with awe, looking in to a room filled with gems and jewels.
6 (Child) is holding a handful of precious stones while (Old Wizard) is watching from afar.
7 (Child) is surprised to see the (Old Wizard) in the forest.
8 (Child) is returning the jewels to the (Old Wizard) in the forest.
9 (Child) is returning the gems to the (Old Wizard) in the forest.
10 (Child) is returning the jewels and gems to the (Old Wizard) in the forest.
11 (Child) i

In [8]:
def prompt_compose(prompt):
    new_prompt = prompt
    for key in dd.keys():
        if f'({key})' in prompt:
            new_prompt += f' {key} is {dd[key]}'
    
    return new_prompt + ' ' + "studio ghibli, pixar and disney animation, 3 d, sharp, rendered in enscape, anime key art by greg rutkowski, bloom, Soft lighting"

n_prompts = [prompt_compose(p) for p in prompts]

for i, p in enumerate(n_prompts):
    print(i, p)

0 (Child) is returning a precious stone to the (Old Wizard) in the forest. Child is The child was a young person, most likely a preteen or early teenager. They had dark brown hair and kind brown eyes. They were wearing a simple outfit with a white t-shirt, jeans and sneakers. Old Wizard is  The old wizard was a mysterious figure with a long white beard and wearing a long robe. He had a kind face and piercing blue eyes that seemed to look right into the child's soul. He had a staff in his hand and was wearing a pointed hat. studio ghibli, pixar and disney animation, 3 d, sharp, rendered in enscape, anime key art by greg rutkowski, bloom, Soft lighting
1 (Child) is standing outside the entrance of a mysterious cave, looking up at an (Old Wizard) standing in the shadows. Child is The child was a young person, most likely a preteen or early teenager. They had dark brown hair and kind brown eyes. They were wearing a simple outfit with a white t-shirt, jeans and sneakers. Old Wizard is  The 

In [21]:
from src.images import stable_diffusion_api as sda

imodel = sda.StableDiffusionApi(save_seed=True, image_formatter=lambda u: Image(url=u[0]))
iconfig = sda.StableDiffusionApiModelConfig()

In [22]:
NEGATIVE_TEMPLATE = (
'low quality, extra fingers, missing fingers',   
'bad anatomy, mutilations, deformations, ugly',
'text, too dark, too sharp, unclear, underexposed' 
'undeveloped, watermark ugly, low quality, missing fingers',
'too many fingers, missing limbs, extra limbs, warped, mutilated', 
'blurry, texts, watermarks, overexposed, too dark',
'disconected limbs, ((((mutated hands and fingers)))), missing limbs' 
'(((fake leg))), contortionist, contorted limbs, (body merged), disjointed' 
'floating limbs, (malformed hands), (((extra limb)), (((extra fingers))), ((bad anatomy))',
'(incomplete face), (scribble), (((disfigured))), dead, ((blurry)), (((incorrect)))',
'((deformed)), (mutated),(mutation), disgusting'
)

neg = ','.join(NEGATIVE_TEMPLATE)

In [23]:
imodel.text2image(n_prompts[0], neg=neg)

......... saving seed 3399391701


In [24]:
imodel.text2image(n_prompts[1], neg=neg)

......... generate by seed 3399391701


In [25]:
imodel.text2image(n_prompts[2], neg=neg)

......... generate by seed 3399391701


In [26]:
imodel.text2image(n_prompts[3], neg=neg)

......... generate by seed 3399391701


In [27]:
imodel.text2image(n_prompts[4], neg=neg)

......... generate by seed 3399391701


In [28]:
imodel.text2image(n_prompts[5], neg=neg)

......... generate by seed 3399391701


In [29]:
imodel.text2image(n_prompts[6], neg=neg)

......... generate by seed 3399391701
